In [35]:
from preprocessors import clean, remove_stopwords, tokenize
import pickle
import os
import csv

import pandas as pd
import numpy as np

import nltk
from sklearn.naive_bayes import GaussianNB

In [4]:
with open('id.stopwords.02.01.2016.txt', 'r') as file:
    csv = csv.reader(file)
    stopwords = []
    for row in csv:
        stopwords += row

In [6]:
tweets = []
for filename in os.listdir('tweet-data/'):
    subdata = pickle.load(open('tweet-data/' + filename, 'rb'))
    tweets = tweets + subdata

In [9]:
def clean_tokens(tweet):
    return remove_stopwords(tokenize(clean(tweet['text'])), stopwords)

def create_fdist(tweets):
    words = []
    for tweet in tweets:
        words += clean_tokens(tweet)
    return nltk.probability.FreqDist(words)

fdist = create_fdist(tweets)
fdist.pprint()
len(fdist)

FreqDist({'yg': 184, 'prabowo': 123, 'presiden': 118, 'jokowi': 115, '@prabowo': 103, '@jokowi': 74, '#jokowi2periode': 53, '#prabowosandi': 53, 'sandi': 50, 'boyolali': 49, ...})


3627

In [31]:
def extract_feature(tweet, features, capres):
    tokens = clean_tokens(tweet)
    fdist = nltk.probability.FreqDist(tokens)
    label = tweet['aspect'][capres]
    return np.hstack(([fdist[f] for f in features], label))

def create_feature_matrix(tweets, features, capres):
    feature_matrix = np.array([]).reshape(0, len(features)+1)
    for tweet in tweets:
        row = extract_feature(tweet, features, capres)
        if row[-1] != 0:
            row[-1] = 1
        feature_matrix = np.vstack((feature_matrix, row))
    return feature_matrix

In [55]:
features = [word for word, count in fdist.most_common(1000)]
feature_matrix_jokowi = create_feature_matrix(tweets, features, 'jokowi')
feature_matrix_prabowo = create_feature_matrix(tweets, features, 'prabowo')

In [56]:
feature_matrix_jokowi

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [2., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [57]:
train_jokowi = feature_matrix_jokowi[:, :-1]
label_jokowi = feature_matrix_jokowi[:, -1:].flatten()

train_prabowo = feature_matrix_prabowo[:, :-1]
label_prabowo = feature_matrix_prabowo[:, -1:].flatten()

In [58]:
nbc_jokowi = GaussianNB()
nbc_jokowi.fit(train_jokowi, label_jokowi)

nbc_prabowo = GaussianNB()
nbc_prabowo.fit(train_prabowo, label_prabowo)

GaussianNB(priors=None)

In [59]:
print(nbc_jokowi.score(train_jokowi, label_jokowi), \
      nbc_prabowo.score(train_prabowo, label_prabowo))

0.8975265017667845 0.8250883392226148


SyntaxError: invalid syntax (<ipython-input-62-57e32467ed32>, line 1)